In [1]:
import threading
import multiprocessing
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.contrib.slim as slim
import scipy.signal
from random import choice
from time import sleep
from time import time
import skimage as skimage
from skimage import transform, color, exposure
from skimage.transform import rotate
import gym
import gym_ple

couldn't import doomish
Couldn't import doom


In [2]:
def update_target_graph(from_scope, to_scope):
    from_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, from_scope)
    to_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, to_scope)

    op_holder = []
    for from_var,to_var in zip(from_vars,to_vars):
        op_holder.append(to_var.assign(from_var))
    return op_holder

In [3]:
def process_frame(frame):
    s = skimage.color.rgb2gray(frame)
    s = scipy.misc.imresize(s,[84,84])
    s = np.reshape(s,[np.prod(s.shape)]) / 255.0
    return s

In [4]:
# Discounting function used to calculate discounted returns.
def discount(x, gamma):
    return scipy.signal.lfilter([1], [1, -gamma], x[::-1], axis=0)[::-1]

#Used to initialize weights for policy and value output layers
def normalized_columns_initializer(std=1.0):
    def _initializer(shape, dtype=None, partition_info=None):
        out = np.random.randn(*shape).astype(np.float32)
        out *= std / np.sqrt(np.square(out).sum(axis=0, keepdims=True))
        return tf.constant(out)
    return _initializer

In [5]:
class AC_Network():
    def __init__(self, s_size, a_size, scope, trainer):
        with tf.variable_scope(scope):
            #Input and visual encoding layers
            self.inputs = tf.placeholder(shape=[None, s_size], dtype=tf.float32)
            self.imageIn = tf.reshape(self.inputs, shape=[-1, 84, 84, 1])
            self.conv1 = slim.conv2d(self.imageIn, 16, [8, 8], stride=[4, 4],
                activation_fn=tf.nn.elu, padding='SAME')
            self.conv2 = slim.conv2d(self.conv1, 32, [4, 4], stride=[2, 2],
                activation_fn=tf.nn.elu, padding='SAME')
            #self.conv3 = slim.conv2d(self.conv2, 64, [3, 3], stride=[1, 1],
            #    activation_fn=tf.nn.elu, padding='SAME')
            hidden = slim.fully_connected(slim.flatten(self.conv2), 256, activation_fn=tf.nn.elu)

            #Recurrent network for temporal dependencies
            lstm_cell = tf.contrib.rnn.BasicLSTMCell(256, state_is_tuple=True)
            c_init = np.zeros((1, lstm_cell.state_size.c), np.float32)
            h_init = np.zeros((1, lstm_cell.state_size.h), np.float32)
            self.state_init = [c_init, h_init]
            c_in = tf.placeholder(tf.float32, [1, lstm_cell.state_size.c])
            h_in = tf.placeholder(tf.float32, [1, lstm_cell.state_size.h])
            self.state_in = (c_in, h_in)
            rnn_in = tf.expand_dims(hidden, [0])
            step_size = tf.shape(self.imageIn)[:1]
            state_in = tf.contrib.rnn.LSTMStateTuple(c_in, h_in)
            lstm_outputs, lstm_state = tf.nn.dynamic_rnn(
                lstm_cell, rnn_in, initial_state=state_in, sequence_length=step_size,
                time_major=False)
            lstm_c, lstm_h = lstm_state
            self.state_out = (lstm_c[:1, :], lstm_h[:1, :])
            rnn_out = tf.reshape(lstm_outputs, [-1, 256])

            #Output layers for policy and value estimations
            self.policy = slim.fully_connected(rnn_out, a_size,
                activation_fn=tf.nn.softmax,
                weights_initializer=normalized_columns_initializer(0.01),
                biases_initializer=None)
            self.value = slim.fully_connected(rnn_out, 1,
                activation_fn=None,
                weights_initializer=normalized_columns_initializer(1.0),
                biases_initializer=None)

            #Only the worker network need ops for loss functions and gradient updating.
            if scope != 'global':
                self.actions = tf.placeholder(shape=[None], dtype=tf.int32)
                self.actions_onehot = tf.one_hot(self.actions, a_size, dtype=tf.float32)
                self.target_v = tf.placeholder(shape=[None], dtype=tf.float32)
                self.advantages = tf.placeholder(shape=[None], dtype=tf.float32)

                self.responsible_outputs = tf.reduce_sum(self.policy * self.actions_onehot, [1])

                #Loss functions
                self.value_loss = 0.5 * tf.reduce_sum(tf.square(self.target_v - tf.reshape(self.value, [-1])))
                self.entropy = - tf.reduce_sum(self.policy * tf.log(self.policy))
                self.policy_loss = -tf.reduce_sum(tf.log(self.responsible_outputs) * self.advantages)
                self.loss = 0.5 * self.value_loss + self.policy_loss - self.entropy * 0.01

                #Get gradients from local network using local losses
                local_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope)
                self.gradients = tf.gradients(self.loss, local_vars)
                self.var_norms = tf.global_norm(local_vars)
                grads, self.grad_norms = tf.clip_by_global_norm(self.gradients, 50.0)

                #Apply local gradients to global network
                global_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'global')
                self.apply_grads = trainer.apply_gradients(zip(grads,global_vars))


In [6]:
class Worker():
    def __init__(self, game, name, s_size, a_size, trainer, model_path, global_episodes):
        self.name = "worker_" + str(name)
        self.number = name
        self.model_path = model_path
        self.trainer = trainer
        self.global_episodes = global_episodes
        self.increment = self.global_episodes.assign_add(1)
        self.episode_rewards = []
        self.episode_lengths = []
        self.episode_mean_values = []
        self.summary_writer = tf.summary.FileWriter("train_" + str(self.number))

        #Create the local copy of the network and the tensorflow op to copy global paramters to local network
        self.local_AC = AC_Network(s_size, a_size, self.name, trainer)
        self.update_local_ops = update_target_graph('global', self.name)

        # Setup the environment
        self.env = gym.make(game)
        self.actions = np.identity(a_size, dtype=bool).tolist()

    def train(self, rollout, sess, gamma, bootstrap_value):
        rollout = np.array(rollout)
        observations = rollout[:,0]
        actions = rollout[:,1]
        rewards = rollout[:,2]
        next_observations = rollout[:,3]
        values = rollout[:,5]

        # Here we take the rewards and values from the rollout, and use them to
        # generate the advantage and discounted returns.
        # The advantage function uses "Generalized Advantage Estimation"
        self.rewards_plus = np.asarray(rewards.tolist() + [bootstrap_value])
        discounted_rewards = discount(self.rewards_plus,gamma)[:-1]
        self.value_plus = np.asarray(values.tolist() + [bootstrap_value])
        advantages = rewards + gamma * self.value_plus[1:] - self.value_plus[:-1]
        advantages = discount(advantages, gamma)

        # Update the global network using gradients from loss
        # Generate network statistics to periodically save
        feed_dict = {self.local_AC.target_v:discounted_rewards,
            self.local_AC.inputs:np.vstack(observations),
            self.local_AC.actions:actions,
            self.local_AC.advantages:advantages,
            self.local_AC.state_in[0]:self.batch_rnn_state[0],
            self.local_AC.state_in[1]:self.batch_rnn_state[1]}
        v_l,p_l,e_l,g_n,v_n, self.batch_rnn_state,_ = sess.run([self.local_AC.value_loss,
            self.local_AC.policy_loss,
            self.local_AC.entropy,
            self.local_AC.grad_norms,
            self.local_AC.var_norms,
            self.local_AC.state_out,
            self.local_AC.apply_grads],
            feed_dict=feed_dict)
        return v_l / len(rollout),p_l / len(rollout),e_l / len(rollout), g_n,v_n

    def work(self, max_episode_length, gamma, sess, coord, saver):
        episode_count = sess.run(self.global_episodes)
        total_steps = 0
        print ("Starting worker " + str(self.number))
        with sess.as_default(), sess.graph.as_default():
            while not coord.should_stop():
                sess.run(self.update_local_ops)
                episode_buffer = []
                episode_values = []
                episode_frames = []
                episode_reward = 0
                episode_step_count = 0
                d = False

                s = self.env.reset()
                episode_frames.append(s)
                s = process_frame(s)
                rnn_state = self.local_AC.state_init
                self.batch_rnn_state = rnn_state

                while d == False:
                    #Take an action using probabilities from policy network output.
                    a_dist, v, rnn_state = sess.run([self.local_AC.policy, self.local_AC.value, self.local_AC.state_out],
                        feed_dict={self.local_AC.inputs:[s],
                        self.local_AC.state_in[0]:rnn_state[0],
                        self.local_AC.state_in[1]:rnn_state[1]})
                    a = np.random.choice(a_dist[0],p=a_dist[0])
                    a = np.argmax(a_dist == a)

                    s1, r, d, info = self.env.step(a)

                    #r = r / 20.0
                    if d == False:
                        episode_frames.append(s1)
                        s1 = process_frame(s1)
                    else:
                        s1 = s

                    episode_buffer.append([s, a, r, s1, d, v[0,0]])
                    episode_values.append(v[0,0])

                    episode_reward += r
                    s = s1
                    total_steps += 1
                    episode_step_count += 1

                    # If the episode hasn't ended, but the experience buffer is full, then we
                    # make an update step using that experience rollout.
                    if len(episode_buffer) == 50 and d != True:
                        # Since we don't know what the true final return is, we "bootstrap" from our current
                        # value estimation.
                        v1 = sess.run(self.local_AC.value,
                            feed_dict={self.local_AC.inputs:[s],
                            self.local_AC.state_in[0]:rnn_state[0],
                            self.local_AC.state_in[1]:rnn_state[1]})[0,0]
                        v_l,p_l,e_l,g_n,v_n = self.train(episode_buffer, sess, gamma, v1)
                        episode_buffer = []
                        sess.run(self.update_local_ops)
                    if d == True:
                        break

                print(self.number, episode_count, episode_reward)
                self.episode_rewards.append(episode_reward)
                self.episode_lengths.append(episode_step_count)
                self.episode_mean_values.append(np.mean(episode_values))

                # Update the network using the experience buffer at the end of the episode.
                if len(episode_buffer) != 0:
                    v_l, p_l, e_l, g_n, v_n = self.train(episode_buffer, sess, gamma, 0.0)


                # Periodically save gifs of episodes, model parameters, and summary statistics.
                if episode_count % 5 == 0 and episode_count != 0:
                    if episode_count % 10 == 0 and self.name == 'worker_0':
                        saver.save(sess,self.model_path+'/model-'+str(episode_count)+'.cptk')
                        print ("Saved Model")

                    mean_reward = np.mean(self.episode_rewards[-5:])
                    mean_length = np.mean(self.episode_lengths[-5:])
                    mean_value = np.mean(self.episode_mean_values[-5:])
                    summary = tf.Summary()
                    summary.value.add(tag='Perf/Reward', simple_value=float(mean_reward))
                    summary.value.add(tag='Perf/Length', simple_value=float(mean_length))
                    summary.value.add(tag='Perf/Value', simple_value=float(mean_value))
                    summary.value.add(tag='Losses/Value Loss', simple_value=float(v_l))
                    summary.value.add(tag='Losses/Policy Loss', simple_value=float(p_l))
                    summary.value.add(tag='Losses/Entropy', simple_value=float(e_l))
                    summary.value.add(tag='Losses/Grad Norm', simple_value=float(g_n))
                    summary.value.add(tag='Losses/Var Norm', simple_value=float(v_n))
                    self.summary_writer.add_summary(summary, episode_count)

                    self.summary_writer.flush()
                if self.name == 'worker_0':
                    sess.run(self.increment)
                episode_count += 1


In [7]:
max_episode_length = 300
gamma = .99 # discount rate for advantage estimation and reward discounting
s_size = 7056 # Observations are greyscale frames of 84 * 84 * 1
a_size = 2 # Agent can move Left, Right, or Fire
load_model = True
ENV_NAME = 'FlappyBird-v0'
model_path = 'model'


In [8]:
import os
model_path = 'model'
if not os.path.exists(model_path):
    os.makedirs(model_path)

In [ ]:
os.makedirs('frames')

In [9]:
tf.reset_default_graph()
config = tf.ConfigProto(allow_soft_placement=True)

In [10]:
with tf.device("/cpu:0"):
    global_episodes = tf.Variable(0, dtype=tf.int32, name='global_episodes', trainable=False)
    trainer = tf.train.AdamOptimizer(learning_rate=1e-4, epsilon=1e-3)
    master_network = AC_Network(s_size, a_size, 'global', None) # Generate global network
    num_workers = multiprocessing.cpu_count() # Set workers ot number of available CPU threads
    workers = []
    # Create worker classes
    for i in range(num_workers):
        workers.append(Worker(ENV_NAME, i, s_size, a_size, trainer, model_path, global_episodes))
    saver = tf.train.Saver(max_to_keep=5)

[2017-11-16 12:18:48,347] Making new env: FlappyBird-v0
[2017-11-16 12:18:49,516] Making new env: FlappyBird-v0
[2017-11-16 12:18:50,507] Making new env: FlappyBird-v0
[2017-11-16 12:18:51,511] Making new env: FlappyBird-v0
[2017-11-16 12:18:52,395] Making new env: FlappyBird-v0
[2017-11-16 12:18:53,427] Making new env: FlappyBird-v0
[2017-11-16 12:18:54,340] Making new env: FlappyBird-v0
[2017-11-16 12:18:55,223] Making new env: FlappyBird-v0


In [ ]:
with tf.Session(config=config) as sess:
    coord = tf.train.Coordinator()
    if load_model == True:
        print ('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(tf.global_variables_initializer())

    # This is where the asynchronous magic happens.
    # Start the "work" process for each worker in a separate thread.
    worker_threads = []
    for worker in workers:
        worker_work = lambda: worker.work(max_episode_length, gamma, sess, coord, saver)
        t = threading.Thread(target=(worker_work))
        t.start()
        sleep(0.5)
        worker_threads.append(t)
    coord.join(worker_threads)

Loading Model...
INFO:tensorflow:Restoring parameters from model\model-20150.cptk


[2017-11-16 12:18:58,912] Restoring parameters from model\model-20150.cptk


Starting worker 0
Starting worker 1
Starting worker 2
Starting worker 3
Starting worker 4
Starting worker 5
Starting worker 6
Starting worker 7
3 20150 -4.0
4 20150 -3.0
6 20150 -1.0
0 20150 6.0
1 20150 6.0
Saved Model
4 20151 3.0
1 20151 1.0
7 20150 12.0
2 20150 15.0
1 20152 0.0
2 20151 -1.0
0 20151 17.0
1 20153 10.0
2 20152 8.0
5 20150 41.0
5 20151 -5.0
0 20152 12.0
6 20151 41.0
7 20151 30.0
4 20152 37.0
6 20152 -4.0
7 20152 -5.0
0 20153 6.0
1 20154 20.0
7 20153 3.0
1 20155 -3.0
0 20154 -3.0
1 20156 -2.0
6 20153 12.0
2 20153 33.0
5 20152 37.0
4 20153 28.0
2 20154 9.0
0 20155 26.0
5 20153 12.0
1 20157 33.0
7 20154 38.0
0 20156 6.0
6 20154 31.0
5 20154 4.0
2 20155 16.0
6 20155 2.0
0 20157 5.0
2 20156 -4.0
3 20151 113.0
4 20154 28.0
2 20157 -4.0
4 20155 -3.0
3 20152 -3.0
2 20158 -5.0
2 20159 -5.0
4 20156 -3.0
0 20158 2.0
5 20155 9.0
6 20156 4.0
0 20159 -4.0
6 20157 1.0
1 20158 22.0
1 20159 -5.0
3 20153 14.0
1 20160 2.0
5 20156 12.0
0 20160 11.0
Saved Model
6 20158 9.0
1 20161 5.0
0 2016

7 20218 21.0
5 20227 27.0
5 20228 2.0
5 20229 -1.0
1 20233 42.0
7 20219 17.0
1 20234 -2.0
0 20247 77.0
2 20219 40.0
6 20228 43.0
3 20220 59.0
6 20229 5.0
7 20220 27.0
6 20230 2.0
2 20220 19.0
5 20230 41.0
3 20221 10.0
5 20231 0.0
5 20232 -4.0
1 20235 41.0
5 20233 7.0
4 20231 102.0
7 20221 31.0
5 20234 13.0
1 20236 25.0
2 20221 38.0
3 20222 36.0
4 20232 16.0
5 20235 6.0
5 20236 -4.0
2 20222 4.0
7 20222 20.0
5 20237 0.0
4 20233 11.0
2 20223 5.0
4 20234 2.0
5 20238 7.0
2 20224 5.0
5 20239 0.0
1 20237 29.0
5 20240 -4.0
7 20223 16.0
7 20224 -3.0
5 20241 -1.0
2 20225 5.0
0 20248 127.0
5 20242 8.0
6 20231 100.0
2 20226 13.0
3 20223 61.0
3 20224 -1.0
6 20232 14.0
3 20225 1.0
1 20238 43.0
0 20249 27.0
7 20225 49.0
1 20239 6.0
6 20233 10.0
4 20235 65.0
0 20250 18.0
Saved Model
1 20240 10.0
2 20227 56.0
2 20228 1.0
6 20234 32.0
0 20251 25.0
3 20226 51.0
1 20241 23.0
2 20229 9.0
2 20230 -3.0
1 20242 23.0
4 20236 67.0
0 20252 25.0
1 20243 -4.0
6 20235 31.0
5 20243 120.0
1 20244 6.0
4 20237 10.0
1 2

2 20297 -1.0
0 20316 19.0
3 20293 9.0
1 20310 3.0
1 20311 6.0
2 20298 12.0
3 20294 9.0
6 20314 21.0
7 20294 36.0
1 20312 5.0
7 20295 0.0
3 20295 9.0
6 20315 8.0
6 20316 -1.0
5 20314 45.0
4 20321 44.0
7 20296 7.0
0 20317 41.0
1 20313 17.0
5 20315 9.0
6 20317 10.0
2 20299 37.0
0 20318 8.0
4 20322 12.0
1 20314 5.0
6 20318 0.0
3 20296 28.0
3 20297 -1.0
2 20300 5.0
0 20319 5.0
2 20301 0.0
3 20298 4.0
7 20297 32.0
0 20320 12.0
3 20299 3.0
Saved Model
2 20302 17.0
5 20316 44.0
7 20298 15.0
2 20303 -2.0
2 20304 -4.0
1 20315 43.0
1 20316 -4.0
1 20317 -1.0
3 20300 26.0
4 20323 57.0
6 20319 56.0
7 20299 15.0
5 20317 19.0
0 20321 32.0
5 20318 -2.0
2 20305 20.0
5 20319 -1.0
7 20300 8.0
4 20324 12.0
1 20318 18.0
0 20322 9.0
4 20325 -4.0
3 20301 15.0
2 20306 5.0
3 20302 -3.0
1 20319 -1.0
5 20320 10.0
5 20321 2.0
6 20320 32.0
1 20320 21.0
3 20303 32.0
3 20304 -4.0
3 20305 3.0
4 20326 59.0
1 20321 29.0
4 20327 -4.0
3 20306 23.0
2 20307 73.0
1 20322 10.0
6 20321 66.0
1 20323 6.0
7 20301 96.0
1 20324 -2.

...

7 20501 35.0
4 20570 1.0
1 20537 0.0
0 20590 6.0
Saved Model
3 20537 19.0
6 20546 37.0
2 20523 10.0
2 20524 2.0
7 20502 21.0
0 20591 23.0
5 20562 56.0
0 20592 1.0
5 20563 13.0
0 20593 12.0
1 20538 49.0
0 20594 -3.0
3 20538 47.0
2 20525 37.0
1 20539 9.0
0 20595 7.0
2 20526 3.0
1 20540 -3.0
3 20539 8.0
3 20540 -4.0
7 20503 47.0
3 20541 -2.0
5 20564 33.0
4 20571 85.0
5 20565 -5.0
6 20547 78.0
2 20527 18.0
6 20548 -1.0
3 20542 13.0
7 20504 17.0
4 20572 7.0
2 20528 1.0
2 20529 -3.0
4 20573 5.0
5 20566 17.0
0 20596 40.0
3 20543 11.0
7 20505 30.0
5 20567 22.0
1 20541 69.0
0 20597 31.0
6 20549 55.0
1 20542 19.0
4 20574 56.0
3 20544 50.0
7 20506 35.0
6 20550 11.0
2 20530 63.0
6 20551 5.0
5 20568 44.0
1 20543 12.0
2 20531 7.0
3 20545 14.0
4 20575 23.0
0 20598 58.0
6 20552 20.0
6 20553 2.0
4 20576 16.0
1 20544 30.0
6 20554 3.0
0 20599 15.0
3 20546 37.0
2 20532 41.0
2 20533 3.0
7 20507 69.0
0 20600 13.0
1 20545 21.0
Saved Model
7 20508 3.0
1 20546 12.0
2 20534 22.0
3 20547 37.0
4 20577 52.0
7 2050

7 20580 20.0
5 20643 -1.0
4 20654 10.0
0 20690 5.0
4 20655 -4.0
Saved Model
4 20656 -1.0
1 20624 10.0
6 20608 4.0
0 20691 1.0
4 20657 -3.0
3 20612 21.0
1 20625 0.0
6 20609 4.0
0 20692 7.0
0 20693 -3.0
0 20694 -1.0
4 20658 16.0
7 20581 31.0
0 20695 0.0
2 20588 56.0
3 20613 20.0
7 20582 -2.0
3 20614 -4.0
5 20644 37.0
0 20696 2.0
7 20583 0.0
6 20610 20.0
4 20659 8.0
0 20697 -3.0
7 20584 3.0
2 20589 12.0
2 20590 -4.0
5 20645 14.0
0 20698 8.0
0 20699 0.0
5 20646 2.0
5 20647 -4.0
3 20615 25.0
7 20585 14.0
2 20591 12.0
3 20616 0.0
0 20700 5.0
Saved Model
5 20648 3.0
7 20586 0.0
4 20660 29.0
7 20587 -4.0
7 20588 -3.0
2 20592 5.0
7 20589 2.0
3 20617 18.0
2 20593 9.0
4 20661 26.0
6 20611 62.0
6 20612 -2.0
0 20701 45.0
2 20594 21.0
3 20618 23.0
4 20662 18.0
0 20702 2.0
0 20703 -2.0
3 20619 6.0
4 20663 11.0
7 20590 56.0
7 20591 -4.0
6 20613 40.0
0 20704 16.0
1 20626 147.0
2 20595 36.0
0 20705 3.0
6 20614 5.0
3 20620 33.0
5 20649 99.0
6 20615 5.0
2 20596 10.0
6 20616 7.0
6 20617 -3.0
3 20621 15.0
6

6 20697 -2.0
6 20698 -3.0
4 20735 35.0
3 20683 29.0
7 20645 5.0
5 20722 16.0
7 20646 2.0
1 20707 27.0
5 20723 1.0
7 20647 0.0
5 20724 -4.0
3 20684 16.0
3 20685 4.0
1 20708 13.0
4 20736 29.0
7 20648 13.0
7 20649 -4.0
4 20737 0.0
3 20686 5.0
1 20709 7.0
5 20725 30.0
6 20699 72.0
2 20663 110.0
5 20726 21.0
1 20710 29.0
1 20711 1.0
6 20700 9.0
3 20687 44.0
4 20738 57.0
4 20739 2.0
2 20664 26.0
2 20665 3.0
0 20781 170.0
5 20727 46.0
0 20782 -1.0
5 20728 1.0
0 20783 0.0
4 20740 27.0
3 20688 47.0
5 20729 4.0
6 20701 54.0
2 20666 27.0
0 20784 9.0
5 20730 5.0
3 20689 11.0
5 20731 -1.0
3 20690 -4.0
3 20691 -3.0
1 20712 73.0
5 20732 0.0
4 20741 28.0
6 20702 21.0
3 20692 3.0
2 20667 29.0
0 20785 34.0
5 20733 20.0
0 20786 2.0
3 20693 24.0
2 20668 17.0
1 20713 36.0
4 20742 37.0
6 20703 38.0
0 20787 15.0
4 20743 -2.0
1 20714 10.0
0 20788 0.0
7 20650 188.0
3 20694 18.0
6 20704 6.0
0 20789 -1.0
5 20734 34.0
7 20651 -2.0
3 20695 -2.0
0 20790 -3.0
Saved Model
6 20705 1.0
4 20744 23.0
2 20669 43.0
7 20652

5 20806 87.0
6 20774 19.0
5 20807 3.0
2 20754 30.0
3 20763 49.0
0 20865 13.0
4 20801 24.0
7 20719 38.0
7 20720 3.0
5 20808 23.0
3 20764 21.0
0 20866 16.0
5 20809 -2.0
7 20721 3.0
5 20810 -3.0
3 20765 0.0
4 20802 24.0
3 20766 2.0
1 20786 141.0
7 20722 7.0
0 20867 18.0
4 20803 5.0
0 20868 -1.0
4 20804 -1.0
7 20723 22.0
0 20869 10.0
3 20767 26.0
7 20724 -3.0
0 20870 -3.0
Saved Model
3 20768 -3.0
1 20787 35.0
4 20805 20.0
6 20775 86.0
6 20776 -1.0
7 20725 20.0
0 20871 17.0
6 20777 0.0
1 20788 16.0
1 20789 -3.0
6 20778 2.0
6 20779 -5.0
0 20872 6.0
1 20790 -1.0
4 20806 27.0
1 20791 2.0
6 20780 8.0
5 20811 91.0
4 20807 5.0
3 20769 49.0
4 20808 2.0
2 20755 137.0
0 20873 26.0
5 20812 14.0
7 20726 45.0
3 20770 16.0
1 20792 30.0
0 20874 13.0
1 20793 3.0
1 20794 -2.0
5 20813 19.0
0 20875 7.0
6 20781 44.0
1 20795 -1.0
2 20756 29.0
5 20814 7.0
3 20771 25.0
7 20727 40.0
3 20772 6.0
4 20809 55.0
0 20876 21.0
3 20773 -3.0
5 20815 12.0
0 20877 -3.0
4 20810 2.0
4 20811 -3.0
1 20796 27.0
6 20782 29.0
0 20

5 20889 12.0
2 20831 6.0
5 20890 2.0
0 20955 19.0
7 20811 4.0
5 20891 -4.0
3 20826 34.0
6 20851 34.0
0 20956 -1.0
6 20852 -4.0
0 20957 -4.0
7 20812 2.0
0 20958 2.0
3 20827 9.0
3 20828 -5.0
0 20959 1.0
2 20832 29.0
5 20892 30.0
2 20833 5.0
1 20849 62.0
1 20850 -1.0
1 20851 -4.0
0 20960 24.0
Saved Model
3 20829 34.0
7 20813 50.0
0 20961 5.0
2 20834 17.0
6 20853 54.0
3 20830 1.0
1 20852 11.0
3 20831 -4.0
1 20853 -4.0
4 20891 101.0
7 20814 5.0
0 20962 8.0
0 20963 -3.0
0 20964 -4.0
1 20854 6.0
3 20832 7.0
4 20892 8.0
0 20965 0.0
1 20855 2.0
2 20835 17.0
7 20815 14.0
1 20856 0.0
7 20816 -3.0
5 20893 62.0
4 20893 18.0
3 20833 22.0
0 20966 19.0
5 20894 6.0
2 20836 20.0
5 20895 -2.0
3 20834 8.0
1 20857 26.0
7 20817 24.0
5 20896 5.0
6 20854 58.0
1 20858 0.0
2 20837 10.0
6 20855 8.0
7 20818 16.0
0 20967 32.0
6 20856 2.0
6 20857 7.0
1 20859 34.0
3 20835 43.0
2 20838 42.0
5 20897 49.0
7 20819 28.0
6 20858 11.0
5 20898 -4.0
5 20899 -3.0
2 20839 5.0
2 20840 -4.0
3 20836 14.0
1 20860 17.0
3 20837 -4.0